# Pauta taller T2b

## IIC2115 - Programación Como Herramienta para la Ingeniería

## Ayudante: Ignacio Rojas López

**Problema:** Programar una función *conteo_subfilas_x*, que cuente el número de subfilas que se pueden obtener a partir de una fila de letras, cuyas primer y última letra sean iguales.

Debe funcionar de tal manera que:

In [ ]:
fila_inicial = "hjdhji"
n_subfilas = conteo_subfilas_x(fila_inicial)
print(n_subfilas)

8


# Esquema A: Iterativa

Atacamos el problema con un "doble for":

In [ ]:
def conteo_subfilas_a(fila):
  if not fila:
    return 0

  subfilas_cnt = len(fila)  # contador de subfilas, parte desde el largo de la fila

  # Recorremos la fila de letras:
  for pos, letra in enumerate(fila):
    # Recorremos el resto de las letras buscando repetidas:
    otras_letras = fila[pos + 1:]
    for otra_letra in otras_letras:
      if letra == otra_letra:
        subfilas_cnt+=1

  return subfilas_cnt      


In [ ]:
if __name__ == '__main__':
  fila_inicial = "hjdhji"
  n_subfilas = conteo_subfilas_a(fila_inicial)
  print(n_subfilas)

8


# Esquema B: Recursiva

Nota: La solución presentada aquí es diferente a la mostrada en la ayudantía del jueves, se modificó debido a que refleja de mejor forma el uso de recursión para resolver del problema

In [ ]:
# Funcion que llamaremos:
def conteo_subfilas_b(fila):
  if not fila:
    return 0
  
  # Nuestra función auxiliar recibe un string "fila", y la posición inicial y final
  # del substring que analizaremos:
  numero_subfilas = conteo_helper_b(fila, 0, len(fila) - 1)
  return numero_subfilas

# Esta función auxiliar nos permite trabajar con más parámetros en nuestra recursión,
# respetando el formato pedido en el enunciado. 
def conteo_helper_b(fila, inicio, final):
  # Casos base
  if (inicio == final): # si el largo es igual a 1, sumamos 1 subfila válida:
    return 1
  if (inicio > final): # si el largo es 0, no sumamos nada:
    return 0

  # Continuamos la recursión disminuyendo nuestra fila de 3 maneras: 
  num_subfilas = (
      conteo_helper_b(fila, inicio + 1, final) # 1. contamos subfilas válidas que se pueden obtener "eliminando" el primer elemento.
      + conteo_helper_b(fila, inicio, final - 1) # 2. contamos subfilas válidas que se pueden obtener "eliminando" el último elemento.
      - conteo_helper_b(fila, inicio + 1, final - 1) # 3. descontamos los casos repetidos de 1. y 2. "eliminando" tanto el primer como último elemento.
      )    
  
  if (fila[inicio] == fila[final]): # Si nuestra fila actual es válida, la contamos
      num_subfilas += 1

  return num_subfilas


In [ ]:
if __name__ == '__main__':
  fila_inicial = "hjdhji"
  n_subfilas = conteo_subfilas_b(fila_inicial)
  print(n_subfilas)

8


## Visualización gráfica de la recursión:

En la siguiente imagen podemos ver visualizado en un grafo lo que ocurre al llamar a nuestra función recursiva <i>*conteo_subfilas_b("abcab")*</i>. Cada nodo corresponde a una llamada a la función realizada por su nodo padre. Podemos ver que el nodo inicial "abcab" genera otros 3 llamados a la función con una fila cada vez más pequeña, eliminando el último elemento (nodo "abca"), eliminando el primero (nodo "bcab"), y eliminando ambos (nodo "bca").

Cuando se llega a los casos base, la función comienza a retornar su valor a su nodo padre. El nodo padre va a sumar lo recibido por los hijos de la izquierda y el medio (números azules), y restar lo del hijo de la derecha (números rojos). Esta resta se realiza para tener en cuenta los casos de subfilas válidas que se repiten en las primeras dos llamadas a la función. Por último, cada nodo revisa si su propia subfila es válida, en cuyo caso suma 1 a lo obtenido de los nodos hijos (números verdes), y entrega el valor a la llamada padre.

<img src="figs/graph.png">

# Esquema C: Dividir y conquistar

Utilizamos dividir y conquistar por medio de backtracking:

In [ ]:
# Funcion que llamaremos:
def conteo_subfilas_c(fila):
  if not fila:
    return 0
  
  # Nuestra función auxiliar, además del número de subfilas válidas, retorna un 
  # diccionario "dict_letras", cuyas llaves son las letras del string "fila",
  # y sus valores son la frecuencia de cada letra en "fila".
  dict_letras, numero_subfilas = conteo_helper_c(fila, 0, len(fila) - 1)
  return numero_subfilas

# Al igual que para el esquema B, utilizamos una función auxiliar:
def conteo_helper_c(fila, inicio, final):
  # Caso base, cuando nuestra subfila es de largo 1, retornamos el diccionario y
  # la cantidad de subfilas válidas para esa única letra:
  if inicio >= final:  
      return {fila[inicio]: 1}, 1
      
  # Para los otros casos, dividimos la fila por la mitad y aplicamos recursión a esas mitades:
  else:
    punto_medio = (inicio + final)//2
    dict_letras_i, num_subfilas_i = conteo_helper_c(
        fila, inicio, punto_medio)  # recursión a la mitad izquierda
    dict_letras_d, num_subfilas_d = conteo_helper_c(
        fila, punto_medio + 1, final)  # recursión a la mitad derecha

    # sumamos la cantidad de subfilas de ambos lados al contador de subfilas.
    cnt_subfilas = num_subfilas_i + num_subfilas_d

    # Si hay letras iguales en la subfila izuierda y derecha, significa que hay nuevas subfilas válidas.
    # Las contamos multiplicado la frecuencia de cada letra de un lado, con la frecuencia de esa letra en el otro lado:
    for letra in dict_letras_i:
        if letra in dict_letras_d:
            cnt_subfilas += dict_letras_i[letra] * dict_letras_d[letra]

    # Unimos ambos diccionarios, para tener una version actualizada:
    for letra in dict_letras_d:
        if letra in dict_letras_i:
            dict_letras_i[letra] += dict_letras_d[letra]
        else:
            dict_letras_i[letra] = dict_letras_d[letra]
    
    # Retornamos el diccionario actualizado y la cantidad de subfilas:
    return dict_letras_i, cnt_subfilas

In [ ]:
if __name__ == '__main__':
  fila_inicial = "hjdhji"
  n_subfilas = conteo_subfilas_c(fila_inicial)
  print(n_subfilas)


8
